In [1]:
import pandas as pd
from functools import reduce
from concurrent.futures import ThreadPoolExecutor
import requests
from urllib.parse import urlencode
import json
import requests
import pandas
import os
from pandas.io.json import json_normalize
import numpy as np

In [2]:
os.environ['Nfer_Access_Key'] = "praveen@nference.net"
os.environ['Nfer_Secret_Key'] = "81d4782ff7cda46fdbd1c2b758af0759"

PRESTAGING_COOKIE = {
"Cookie":"csrftoken=wLgolDsSbcb3Py2xB8xuiRDmwezQ9Y9o; sessionid=jqab5fdwqmrxrnc5u0cli8xqxerep2p2"}

headers = {}
headers["cookie"] = "csrftoken=wLgolDsSbcb3Py2xB8xuiRDmwezQ9Y9o; sessionid=jqab5fdwqmrxrnc5u0cli8xqxerep2p2" 

In [3]:
url = "https://preview.nferx.com/dv/201912/rnaapi2/entity_distribution?"

#https://preview.nferx.com/rnaapi2/entity_distribution?hgnc_id=il1a&gene_id=ensg00000115008&range_a_min=0&range_a_max=0.07&range_b_min=3.86&range_b_max=14&entity_type=root-cells-and-tissues&enriched_group=B&cell_line_filter=not_cell_line&single_cell_filter=not_single_cell&exclude_zero=true&token_extraction=ngram

params = {
    "entity_type":"root-cells-and-tissues",
    "range_a_min":0,
    "range_a_max":0.07,
    "range_b_min":3.86,
    "range_b_max":14,
    "enriched_group": "B",
    "exclude_zero": True,
    "token_extraction": "ngram",
    "single_cell_filter":"not_single_cell",
    "cell_line_filter":"all",
    "exclude_zero":True,
    "gene_id":"ensg00000115008",
    "hgnc_id":"IL1A"
}

x = requests.get(url, params=params, headers = headers)

In [4]:
json_normalize(x.json()['result']['term_features'])

,neg_log_p_bh_adjusted_p,term,range_a_count,range_b_count,all_studies_count,gtex_studies_count,tcga_studies_count,signal_token,annotation,ls,gs
0,0.263398,muscle,232,42,1,0,1,muscle,[tcga_evidence],1.47,0.0
1,1.549081,o_cells,4,4,1,0,0,o_cells,[],0.00,0.0
2,4.623567,epithelium_cell,0,6,1,0,0,epithelium_cell,[],0.00,0.0
3,12.767115,human_coronary_artery_smooth_muscle_cells,0,16,2,0,0,human_coronary_artery_smooth_muscle_cells,[],0.00,0.0
4,23.515067,mcf10dcis_com,0,29,2,0,0,mcf10dcis_com,[],0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
341,1.483422,breast_epithelial_cell,0,2,1,0,0,breast_epithelial_cell,[],0.00,0.0
342,0.711467,efo21,0,1,1,0,0,efo21,[],0.00,0.0
343,0.711467,lm2_4175,0,1,1,0,0,lm2_4175,[],0.00,0.0
344,1.483422,tc7_cells,0,2,1,0,0,tc7_cells,[],0.00,0.0


In [4]:
def get_enrichment(hgnc_id,ensemble_id,lowrange,highrange,entity):
    url = "https://preview.nferx.com/dv/201912/rnaapi2/entity_distribution?"
    
    params = {
        "entity_type":entity,
        "range_a_min":lowrange[0],
        "range_a_max":lowrange[1],
        "range_b_min":highrange[0],
        "range_b_max":highrange[1],
        "enriched_group": "B",
        "exclude_zero": True,
        "token_extraction": "ngram",
        "single_cell_filter":"not_single_cell",
        "cell_line_filter":"not_cell_line",
        "exclude_zero":True,
        "gene_id":ensemble_id,
        "hgnc_id":hgnc_id
    }

    x = requests.get(url, params=params, headers = headers)
    required_table = json_normalize(x.json()['result']['term_features'])
    return(required_table)

In [5]:
def get_enrichment_low(hgnc_id,ensemble_id,lowrange,highrange,entity):
    url = "https://preview.nferx.com/dv/201912/rnaapi2/entity_distribution?"
    
    params = {
        "entity_type":entity,
        "range_a_min":lowrange[0],
        "range_a_max":lowrange[1],
        "range_b_min":highrange[0],
        "range_b_max":highrange[1],
        "enriched_group": "A",
        "exclude_zero": True,
        "token_extraction": "ngram",
        "single_cell_filter":"not_single_cell",
        "cell_line_filter":"not_cell_line",
        "exclude_zero":True,
        "gene_id":ensemble_id,
        "hgnc_id":hgnc_id
    }

    x = requests.get(url, params=params, headers = headers)
    required_table = json_normalize(x.json()['result']['term_features'])
    return(required_table)

In [7]:
all_IL1A_celltissue_enriched = get_enrichment("il1a","ensg00000115008",[0,0.07],[3.86,14],"root-cells-and-tissues")
all_IL1A_disease_enriched = get_enrichment("il1a","ensg00000115008",[0,0.07],[3.86,14],"root-diseases")

In [8]:
all_IL1A_celltissue_enriched.shape

(151, 11)

In [9]:
all_IL1B_celltissue_enriched = get_enrichment("IL1B","ensg00000125538",[0,0.46],[5.68,14],"root-cells-and-tissues")
all_IL1B_disease_enriched = get_enrichment("IL1B","ensg00000125538",[0,0.46],[5.68,14],"root-diseases")

In [10]:
all_IL1B_disease_enriched.shape

(75, 11)

In [11]:
all_IL1B_disease_enriched

,neg_log_p_bh_adjusted_p,term,range_a_count,range_b_count,all_studies_count,gtex_studies_count,tcga_studies_count,signal_token,annotation,ls,gs
0,30.061461,acute_on_chronic_liver_failure,0,40,1,0,0,acute_on_chronic_liver_failure,[],0.00,0.0
1,0.588063,fibrosarcoma,0,1,1,0,0,fibrosarcoma,[],0.78,0.0
2,36.202791,myelofibrosis,0,48,1,0,0,myelofibrosis,[],0.90,0.0
3,0.189523,rdeb,4,1,1,0,0,rdeb,[],0.00,0.0
4,0.588063,raeb_1,0,1,1,0,0,raeb_1,[],0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
70,2.594601,tuberculous_meningitis,0,4,1,0,0,tuberculous_meningitis,[],0.10,0.0
71,1.947562,hyper_igd_syndrome,0,3,1,0,0,hyper_igd_syndrome,[],0.39,0.0
72,4.045956,limited_systemic_sclerosis,0,6,1,0,0,limited_systemic_sclerosis,[],0.00,0.0
73,53.819552,lyme_disease,0,71,2,0,0,lyme_disease,[],1.04,0.0


In [23]:
all_IL1R1_celltissue_enriched = get_enrichment("IL1R1","ensg00000115594",[0,0.4],[4.35,14],"root-cells-and-tissues")
all_IL1R1_disease_enriched = get_enrichment("IL1R1","ensg00000115594",[0,0.4],[4.35,14],"root-diseases")

In [25]:
all_IL1R1_disease_enriched.shape

(149, 11)

In [26]:
all_IL1RN_celltissue_enriched = get_enrichment("IL1RN","ensg00000136689",[0,0.35],[5.57,14],"root-cells-and-tissues")
all_IL1RN_disease_enriched = get_enrichment("IL1RN","ensg00000136689",[0,0.35],[5.57,14],"root-diseases")

In [28]:
all_IL1RN_disease_enriched.shape

(86, 11)

In [12]:
all_IL1RN_celltissue_lowenriched = get_enrichment_low("IL1RN","ensg00000136689",[0,0.35],[5.57,14],"root-cells-and-tissues")
all_IL1RN_disease_lowenriched = get_enrichment_low("IL1RN","ensg00000136689",[0,0.35],[5.57,14],"root-diseases")

In [31]:
all_IL1RN_disease_lowenriched.shape

(418, 11)

In [33]:
all_IL1A_celltissue_enriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1A_RNAseq_celltissue_enrichment.csv', index=False)
all_IL1A_disease_enriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1A_RNAseq_disease_enrichment.csv', index=False)


In [34]:
all_IL1B_celltissue_enriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1B_RNAseq_celltissue_enrichment.csv', index=False)
all_IL1B_disease_enriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1B_RNAseq_disease_enrichment.csv', index=False)

all_IL1R1_celltissue_enriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1R1_RNAseq_celltissue_enrichment.csv', index=False)
all_IL1R1_disease_enriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1R1_RNAseq_disease_enrichment.csv', index=False)

all_IL1RN_celltissue_enriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1RN_RNAseq_celltissue_enrichment.csv', index=False)
all_IL1RN_disease_enriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1RN_RNAseq_disease_enrichment.csv', index=False)

In [14]:
all_IL1RN_celltissue_lowenriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1RN_RNAseq_celltissue_low_enrichment.csv', index=False)
all_IL1RN_disease_lowenriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/IL1RN_RNAseq_disease_low_enrichment.csv', index=False)

In [6]:
all_IL1B_disease_lowenriched = get_enrichment_low("IL1B","ensg00000125538",[0,0.35],[6.09,14],"root-diseases")

In [8]:
all_IL1B_disease_lowenriched.to_csv('/Users/praveenanand/Documents/Apps/Bermekimab_LabelExpansen_Janssen_May2020/Deliverable2_again/Data/aug5th_datatransfer/IL1B_RNAseq_disease_low_enrichment.csv', index=False)


In [ ]:
all_IL1A_disease_lowenriched = get_enrichment_low("IL1A","ensg00000115008",[0,0.07],[3.86,14],"root-cells-and-tissues")